<a href="https://colab.research.google.com/github/LouisLiu2000/LOUIS-GPT/blob/main/1_second_level_machine_learning_frame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [3]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

kacobe_1_s_kline_path = kagglehub.dataset_download('kacobe/1-s-kline')

print('Data source import complete.')


100%|██████████| 5.65G/5.65G [02:39<00:00, 38.1MB/s]

Extracting files...


Data source import complete.


In [ ]:
import pandas as pd
import os
import glob
import kagglehub
from datetime import datetime

# ============================================
# 专门合并BTCUSDT 1秒K线数据
# ============================================

# 下载数据集
print("正在获取数据路径...")
try:
    data_path = kagglehub.dataset_download('kacobe/1-s-kline')
    print(f'数据路径: {data_path}')
except Exception as e:
    print(f"获取路径失败: {e}")
    data_path = None

if data_path and os.path.exists(data_path):
    # 递归查找所有符合格式的1秒K线文件
    def find_1s_kline_files(directory):
        files = []
        for root, dirs, file_list in os.walk(directory):
            for file in file_list:
                # 只要包含 "BTCUSDT_1s" 的CSV文件
                if file.startswith('BTCUSDT_1s_') and file.endswith('.csv'):
                    files.append(os.path.join(root, file))
        return sorted(files)

    # 查找所有1秒K线文件
    kline_1s_files = find_1s_kline_files(data_path)

    print(f"\n找到 {len(kline_1s_files)} 个1秒K线文件:")
    for i, file in enumerate(kline_1s_files):
        filename = os.path.basename(file)
        print(f"  {i+1:2d}. {filename}")

    if not kline_1s_files:
        print("❌ 未找到任何BTCUSDT_1s_*.csv文件！")
        print("让我们看看目录中都有什么文件...")

        # 列出所有CSV文件看看格式
        all_csv = []
        for root, dirs, file_list in os.walk(data_path):
            for file in file_list:
                if file.endswith('.csv'):
                    all_csv.append(file)

        print(f"\n目录中的所有CSV文件 (前20个):")
        for i, file in enumerate(all_csv[:20]):
            print(f"  {i+1}. {file}")

    else:
        # 开始合并数据
        print(f"\n开始合并 {len(kline_1s_files)} 个文件...")

        all_dfs = []
        failed_files = []
        total_rows = 0

        for i, file in enumerate(kline_1s_files):
            try:
                filename = os.path.basename(file)
                print(f"📁 [{i+1:2d}/{len(kline_1s_files)}] 读取: {filename}")

                # 读取CSV文件
                df = pd.read_csv(file)

                if len(df) > 0:
                    all_dfs.append(df)
                    total_rows += len(df)

                    # 显示第一个文件的结构信息
                    if i == 0:
                        print(f"    ✅ 列名: {list(df.columns)}")
                        print(f"    ✅ 数据示例:")
                        print(df.head(2).to_string())

                    print(f"    ✅ 成功读取 {len(df):,} 行数据")
                else:
                    print(f"    ⚠️  文件为空")

            except Exception as e:
                print(f"    ❌ 读取失败: {e}")
                failed_files.append(file)

        print(f"\n📊 读取统计:")
        print(f"✅ 成功: {len(all_dfs)} 个文件")
        print(f"❌ 失败: {len(failed_files)} 个文件")
        print(f"📈 总行数: {total_rows:,}")

        if failed_files:
            print(f"\n失败的文件:")
            for f in failed_files:
                print(f"  {os.path.basename(f)}")

        # 合并数据
        if all_dfs:
            print(f"\n🔄 正在合并数据...")

            # 使用concat合并所有数据框
            df_full = pd.concat(all_dfs, ignore_index=True)

            print(f"✅ 合并完成!")
            print(f"📊 总数据行数: {len(df_full):,}")
            print(f"📋 数据列: {list(df_full.columns)}")

            # 处理时间列
            time_columns = [col for col in df_full.columns if 'time' in col.lower()]
            if time_columns:
                print(f"\n⏰ 处理时间数据...")

                # 通常第一个时间列是开盘时间
                time_col = time_columns[0]
                print(f"   使用 '{time_col}' 作为排序依据")

                try:
                    # 转换为datetime
                    df_full[time_col] = pd.to_datetime(df_full[time_col])

                    # 按时间排序
                    df_full = df_full.sort_values(time_col)
                    df_full = df_full.reset_index(drop=True)

                    # 显示时间范围
                    time_range_start = df_full[time_col].min()
                    time_range_end = df_full[time_col].max()
                    print(f"   📅 时间范围: {time_range_start} 到 {time_range_end}")

                    # 计算数据跨度
                    duration = time_range_end - time_range_start
                    print(f"   📏 数据跨度: {duration.days} 天")

                    # 转换其他时间列
                    for col in time_columns[1:]:
                        try:
                            df_full[col] = pd.to_datetime(df_full[col])
                        except:
                            pass

                except Exception as e:
                    print(f"   ⚠️  时间处理警告: {e}")

            # 数据去重（基于时间）
            if time_columns:
                original_len = len(df_full)
                df_full = df_full.drop_duplicates(subset=[time_columns[0]], keep='first')
                removed_duplicates = original_len - len(df_full)
                if removed_duplicates > 0:
                    print(f"🧹 移除重复数据: {removed_duplicates:,} 行")
                    print(f"📊 最终数据行数: {len(df_full):,}")

            # 显示数据预览
            print(f"\n📋 数据预览 (前5行):")
            print(df_full.head())

            # 显示基本统计信息
            if 'open' in df_full.columns:
                print(f"\n📈 价格统计 (开盘价):")
                print(f"   最高: ${df_full['open'].max():.2f}")
                print(f"   最低: ${df_full['open'].min():.2f}")
                print(f"   平均: ${df_full['open'].mean():.2f}")

            # 内存使用情况
            memory_mb = df_full.memory_usage(deep=True).sum() / 1024**2
            print(f"\n💾 内存使用: {memory_mb:.2f} MB")

            print(f"\n🎉 数据合并完成！变量名: df_full")
            print(f"   可以开始进行数据分析了！")

        else:
            print(f"\n❌ 没有成功读取任何数据文件！")

else:
    print(f"❌ 无法访问数据目录，请检查数据下载状态")

In [ ]:
!pip install numba

In [ ]:
import pandas as pd
import numpy as np
from numba import jit
import warnings
import time
warnings.filterwarnings('ignore')

@jit(nopython=True)
def calculate_scores_two_phase(
    close_prices, take_profit, stop_loss, max_holding_time,
    min_signal_interval, max_signals_per_window, window_size,
    time_decay_lambda, near_stop_threshold,
    quick_profit_time, quick_profit_multiplier,
    momentum_penalty_alpha, is_long=True
):
    """
    两阶段评分系统：
    第一阶段 - 客观计算每个点的交易结果和基础分数
    第二阶段 - 全局优化选择信号，应用约束和动量惩罚
    """
    data_length = len(close_prices)

    # ========== 第一阶段：计算所有基础分数 ==========
    base_scores = np.zeros(data_length)
    trade_results = np.zeros(data_length)
    holding_times = np.zeros(data_length)
    max_profits = np.zeros(data_length)
    min_drawdowns = np.zeros(data_length)

    # 有效计算范围
    valid_length = data_length - max_holding_time

    for i in range(valid_length):
        entry_price = close_prices[i]

        if is_long:
            tp_price = entry_price * (1 + take_profit)
            sl_price = entry_price * (1 - stop_loss)
        else:
            tp_price = entry_price * (1 - take_profit)
            sl_price = entry_price * (1 + stop_loss)

        # 交易模拟
        hit_tp = False
        hit_sl = False
        max_profit_seen = 0.0
        min_drawdown_seen = 0.0

        for j in range(i + 1, min(i + 1 + max_holding_time, data_length)):
            current_price = close_prices[j]

            # 记录最大盈亏
            if is_long:
                profit = (current_price - entry_price) / entry_price
                max_profit_seen = max(max_profit_seen, profit)
                min_drawdown_seen = min(min_drawdown_seen, profit)

                # 使用close价格判断止盈止损
                if current_price <= sl_price:
                    hit_sl = True
                    holding_times[i] = j - i
                    break
                elif current_price >= tp_price:
                    hit_tp = True
                    holding_times[i] = j - i
                    break
            else:
                profit = (entry_price - current_price) / entry_price
                max_profit_seen = max(max_profit_seen, profit)
                min_drawdown_seen = min(min_drawdown_seen, profit)

                if current_price >= sl_price:
                    hit_sl = True
                    holding_times[i] = j - i
                    break
                elif current_price <= tp_price:
                    hit_tp = True
                    holding_times[i] = j - i
                    break

        # 记录路径信息
        max_profits[i] = max_profit_seen
        min_drawdowns[i] = min_drawdown_seen

        # 计算基础分数
        if hit_sl:
            trade_results[i] = 1
            base_scores[i] = 0  # 止损直接0分

        elif hit_tp:
            trade_results[i] = 2
            tp_time = holding_times[i]

            # 基础分60-80，根据速度调整
            time_efficiency = np.exp(-time_decay_lambda * (tp_time / max_holding_time))
            score = 60.0 + 20.0 * time_efficiency

            # 快速止盈奖励
            if tp_time <= quick_profit_time:
                score = min(100.0, score * quick_profit_multiplier)

            # 路径风险惩罚（曾接近止损）
            if abs(min_drawdown_seen) > stop_loss * near_stop_threshold:
                score *= 0.8

            base_scores[i] = score

        else:  # 超时
            trade_results[i] = 3
            holding_times[i] = max_holding_time

            # 根据最大盈利给分
            if max_profit_seen > 0:
                # 达到的止盈比例
                tp_ratio = max_profit_seen / take_profit
                score = 10.0 + min(30.0, tp_ratio * 30.0)

                # 如果曾接近止损，额外惩罚
                if abs(min_drawdown_seen) > stop_loss * 0.5:
                    score *= 0.7
            else:
                score = 5.0  # 亏损超时给最低分

            base_scores[i] = score

    # ========== 第二阶段：应用约束选择信号 ==========
    final_scores = base_scores.copy()
    is_selected = np.zeros(data_length, dtype=np.bool_)

    # 找出所有高分候选（>=70）
    high_score_mask = base_scores >= 70
    high_score_count = np.sum(high_score_mask)

    if high_score_count > 0:
        # 获取高分索引
        high_score_indices = np.zeros(high_score_count, dtype=np.int64)
        idx_count = 0
        for i in range(data_length):
            if high_score_mask[i]:
                high_score_indices[idx_count] = i
                idx_count += 1

        # 按分数排序（降序）
        scores_at_indices = np.zeros(high_score_count)
        for i in range(high_score_count):
            scores_at_indices[i] = base_scores[high_score_indices[i]]

        # 简单的冒泡排序（因为numba不支持argsort的fancy indexing）
        for i in range(high_score_count - 1):
            for j in range(high_score_count - 1 - i):
                if scores_at_indices[j] < scores_at_indices[j + 1]:
                    # 交换分数
                    temp_score = scores_at_indices[j]
                    scores_at_indices[j] = scores_at_indices[j + 1]
                    scores_at_indices[j + 1] = temp_score
                    # 交换索引
                    temp_idx = high_score_indices[j]
                    high_score_indices[j] = high_score_indices[j + 1]
                    high_score_indices[j + 1] = temp_idx

        # 贪心选择
        selected_count = 0
        selected_indices = np.zeros(high_score_count, dtype=np.int64)

        for i in range(high_score_count):
            idx = high_score_indices[i]
            can_select = True

            # 检查最小间隔约束
            for j in range(selected_count):
                if abs(idx - selected_indices[j]) < min_signal_interval:
                    can_select = False
                    break

            # 检查窗口约束
            if can_select:
                window_start = max(0, idx - window_size)
                window_end = min(data_length, idx + window_size)
                window_count = 0

                for j in range(selected_count):
                    if window_start <= selected_indices[j] < window_end:
                        window_count += 1

                if window_count >= max_signals_per_window:
                    can_select = False

            # 如果满足约束，选择此信号
            if can_select:
                selected_indices[selected_count] = idx
                selected_count += 1
                is_selected[idx] = True
            else:
                # 不满足约束的高分信号降为0分
                final_scores[idx] = 0

        # 应用动量惩罚到选中的信号
        for i in range(selected_count):
            idx = selected_indices[i]

            # 计算此信号前的密度
            recent_window = max(0, idx - window_size)
            recent_count = 0

            for j in range(i):  # 只看之前选中的信号
                if selected_indices[j] >= recent_window:
                    recent_count += 1

            if recent_count > 0:
                # 动量惩罚
                momentum_penalty = 1.0 / (1.0 + momentum_penalty_alpha * np.log1p(recent_count))
                final_scores[idx] *= momentum_penalty
                final_scores[idx] = min(100.0, max(0.0, final_scores[idx]))

    # 统计信息
    total_base_high = np.sum(high_score_mask)
    total_selected = np.sum(is_selected)
    filtered_count = total_base_high - total_selected

    stats = np.array([
        total_base_high,      # 基础高分信号数
        filtered_count,       # 被过滤的信号数
        total_selected,       # 最终选中的信号数
        0.0,                  # 预留
        0.0                   # 预留
    ], dtype=np.float64)

    return final_scores, base_scores, trade_results, holding_times, is_selected, stats, max_profits, min_drawdowns


def calculate_realistic_scores_optimized(df_original, params, direction='long'):
    """
    使用优化的两阶段评分系统
    """
    df_scored = df_original.copy()

    # 提取参数
    take_profit = params['take_profit'] / 100
    stop_loss = params['stop_loss'] / 100
    max_holding_time = params['max_holding_time']
    min_signal_interval = params.get('min_signal_interval', 60)
    max_signals_per_window = params.get('max_signals_per_window', 3)
    window_size = params.get('window_size', 600)
    time_decay_lambda = params['time_decay_lambda']
    near_stop_threshold = params['near_stop_threshold']
    quick_profit_time = params['quick_profit_bonus_time']
    quick_profit_multiplier = params['quick_profit_bonus_multiplier']
    momentum_penalty_alpha = params['momentum_penalty_alpha']

    print(f"\n{'='*60}")
    print(f"开始计算{direction}方向评分（两阶段优化版）")
    print(f"{'='*60}")
    print(f"数据长度: {len(df_scored):,}")
    print(f"止盈: {take_profit*100:.2f}%, 止损: {stop_loss*100:.2f}%")
    print(f"最大持仓: {max_holding_time}秒")
    print(f"约束: 最小间隔{min_signal_interval}秒")
    print(f"      {window_size}秒窗口内最多{max_signals_per_window}个信号")

    # 提取价格数组
    close_prices = df_scored['close'].values.astype(np.float64)

    # 检查数据质量
    nan_count = np.sum(np.isnan(close_prices))
    if nan_count > 0:
        print(f"⚠️ 警告：发现{nan_count}个NaN值，将被替换为前值")
        # 前向填充NaN
        for i in range(1, len(close_prices)):
            if np.isnan(close_prices[i]):
                close_prices[i] = close_prices[i-1]

    # 计算评分
    is_long = (direction == 'long')

    print("\n正在计算评分...")
    start_time = time.time()

    results = calculate_scores_two_phase(
        close_prices, take_profit, stop_loss, max_holding_time,
        min_signal_interval, max_signals_per_window, window_size,
        time_decay_lambda, near_stop_threshold,
        quick_profit_time, quick_profit_multiplier,
        momentum_penalty_alpha, is_long
    )

    final_scores, base_scores, trade_results, holding_times, is_selected, stats, max_profits, min_drawdowns = results

    end_time = time.time()
    print(f"计算完成，耗时: {end_time - start_time:.2f}秒")

    # 添加结果列
    prefix = f'{direction}_'
    df_scored[f'{prefix}score'] = final_scores
    df_scored[f'{prefix}base_score'] = base_scores
    df_scored[f'{prefix}trade_result'] = trade_results
    df_scored[f'{prefix}holding_time'] = holding_times
    df_scored[f'{prefix}selected'] = is_selected
    df_scored[f'{prefix}max_profit'] = max_profits * 100  # 转换为百分比
    df_scored[f'{prefix}min_drawdown'] = min_drawdowns * 100

    # 解析统计信息
    total_base_high = int(stats[0])
    filtered_count = int(stats[1])
    selected_count = int(stats[2])

    # 计算有效长度
    valid_length = len(df_scored) - max_holding_time
    valid_scores = final_scores[:valid_length]

    print(f"\n{'='*50}")
    print(f"评分统计")
    print(f"{'='*50}")
    print(f"有效评分数量: {valid_length:,}")
    print(f"平均分: {np.mean(valid_scores):.2f}")
    print(f"最高分: {np.max(valid_scores):.2f}")
    print(f"标准差: {np.std(valid_scores):.2f}")

    # 分数分布
    print(f"\n分数分布:")
    print(f"  0分(止损/被过滤): {np.sum(valid_scores == 0):,} ({np.sum(valid_scores == 0)/valid_length*100:.2f}%)")
    print(f"  1-59分: {np.sum((valid_scores > 0) & (valid_scores < 60)):,} ({np.sum((valid_scores > 0) & (valid_scores < 60))/valid_length*100:.2f}%)")
    print(f"  60-69分: {np.sum((valid_scores >= 60) & (valid_scores < 70)):,} ({np.sum((valid_scores >= 60) & (valid_scores < 70))/valid_length*100:.2f}%)")
    print(f"  70-79分: {np.sum((valid_scores >= 70) & (valid_scores < 80)):,} ({np.sum((valid_scores >= 70) & (valid_scores < 80))/valid_length*100:.2f}%)")
    print(f"  80-89分: {np.sum((valid_scores >= 80) & (valid_scores < 90)):,} ({np.sum((valid_scores >= 80) & (valid_scores < 90))/valid_length*100:.2f}%)")
    print(f"  90-100分: {np.sum(valid_scores >= 90):,} ({np.sum(valid_scores >= 90)/valid_length*100:.2f}%)")

    # 交易结果分布
    print(f"\n交易结果分布:")
    tp_count = np.sum(trade_results[:valid_length] == 2)
    sl_count = np.sum(trade_results[:valid_length] == 1)
    timeout_count = np.sum(trade_results[:valid_length] == 3)

    print(f"  止盈: {tp_count:,} ({tp_count/valid_length*100:.2f}%)")
    print(f"  止损: {sl_count:,} ({sl_count/valid_length*100:.2f}%)")
    print(f"  超时: {timeout_count:,} ({timeout_count/valid_length*100:.2f}%)")

    # 约束过滤统计
    print(f"\n约束过滤统计:")
    print(f"  基础高分信号(>=70): {total_base_high:,} ({total_base_high/valid_length*100:.4f}%)")
    print(f"  最终选中的信号: {selected_count:,} ({selected_count/valid_length*100:.4f}%)")
    print(f"  被过滤掉: {filtered_count:,}")
    if total_base_high > 0:
        print(f"  过滤比例: {filtered_count/total_base_high*100:.2f}%")
        print(f"  选择率: {selected_count/total_base_high*100:.2f}%")

    # 验证约束
    selected_indices = np.where(is_selected[:valid_length])[0]
    if len(selected_indices) > 1:
        print(f"\n约束验证:")

        # 验证间隔
        intervals = np.diff(selected_indices)
        min_interval_actual = np.min(intervals)
        avg_interval = np.mean(intervals)

        print(f"  信号间隔:")
        print(f"    最小: {min_interval_actual}秒")
        print(f"    平均: {avg_interval:.1f}秒")
        print(f"    中位数: {np.median(intervals):.1f}秒")

        violations = np.sum(intervals < min_signal_interval)
        if violations > 0:
            print(f"    ⚠️ 发现{violations}个间隔违规！")
        else:
            print(f"    ✓ 所有间隔满足{min_signal_interval}秒约束")

        # 验证窗口约束
        window_violations = 0
        for i, idx in enumerate(selected_indices[:min(100, len(selected_indices))]):
            window_start = max(0, idx - window_size)
            window_end = min(valid_length, idx + window_size)
            window_count = np.sum((selected_indices >= window_start) &
                                 (selected_indices < window_end) &
                                 (selected_indices != idx))
            if window_count >= max_signals_per_window:
                window_violations += 1

        if window_violations == 0:
            print(f"    ✓ 窗口约束验证通过")
        else:
            print(f"    ⚠️ {window_violations}个位置违反窗口约束")

    # 显示样本
    print(f"\n高分信号样本（前10个）:")
    high_score_samples = df_scored[df_scored[f'{prefix}selected'] == True].head(10)
    if len(high_score_samples) > 0:
        display_cols = ['close', f'{prefix}score', f'{prefix}base_score',
                       f'{prefix}holding_time', f'{prefix}max_profit', f'{prefix}min_drawdown']
        print(high_score_samples[display_cols].to_string())

    return df_scored


# ============================================
# 使用示例
# ============================================

if __name__ == "__main__":
    print("="*60)
    print("两阶段优化交易评分系统")
    print("="*60)

    # 准备数据 - 假设df_full已经存在
    # df_full = pd.read_csv('your_data.csv')

    # 清理数据
    df_clean = df_full.copy()
    if 'ignore' in df_clean.columns:
        df_clean = df_clean.drop('ignore', axis=1)
        print("已删除ignore列")

    # 针对1秒K线优化的参数
    scoring_params = {
        # 基础交易参数
        'take_profit': 0.3,              # 0.3%止盈
        'stop_loss': 0.9,                # 0.9%止损
        'max_holding_time': 3600,        # 最大持仓1小时

        # 约束参数（1秒K线需要更严格）
        'min_signal_interval': 60,       # 最小60秒间隔
        'max_signals_per_window': 3,     # 10分钟内最多3个信号
        'window_size': 600,              # 10分钟窗口

        # 评分参数
        'time_decay_lambda': 2.0,        # 时间衰减参数
        'momentum_penalty_alpha': 0.2,   # 动量惩罚系数
        'near_stop_threshold': 0.7,      # 70%止损位置算接近
        'quick_profit_bonus_time': 60,   # 60秒内止盈加分
        'quick_profit_bonus_multiplier': 1.3  # 快速止盈奖励倍数
    }

    # 计算评分
    df_with_scores = calculate_realistic_scores_optimized(
        df_clean,
        scoring_params,
        direction='long'
    )

    # 如果需要计算short方向
    # df_with_scores = calculate_realistic_scores_optimized(
    #     df_with_scores,
    #     scoring_params,
    #     direction='short'
    # )

    print("\n" + "="*60)
    print("评分完成！")
    print("="*60)

完全修复的实际交易打分系统
已删除ignore列
开始计算long方向评分...
数据长度: 167,650,296
止盈: 0.30%, 止损: 0.90%
约束: 最小间隔180秒, 1800秒窗口内最多3个信号
正在计算评分...

===== 评分完成 =====
有效评分数量: 167,563,896
平均分: 3.20
最高分: 96.00
标准差: 13.59

===== 分数分布 =====
0分(止损/被过滤): 158,118,612 (94.36%)
1-59分: 3,575,543 (2.13%)
60-69分: 5,649,973 (3.37%)
70-79分: 189,627 (0.11%)
80-89分: 29,293 (0.02%)
90-100分: 848 (0.00%)

===== 约束过滤统计 =====
基础高分信号(>=70): 117,214,024 (69.9518%)
最终保留的高分信号: 219,768 (0.1312%)
总共过滤掉: 116,994,256
  - 间隔违规: 35,088,835
  - 窗口数量违规: 66,863,709
  - 动量惩罚后分数不足: 15,041,712
过滤比例: 99.81%

===== 一致性验证 =====
统计的保留数量: 219,768
实际保留数量: 219,768
✓ 数量一致

===== 详细约束验证 =====
最终高分信号数(score>=70): 219,768
标记为保留的高分信号数: 219,768

保留信号的间隔统计:
  最小: 180秒
  平均: 762.5秒
  中位数: 498.0秒
  最大: 96955秒
  ✓ 所有保留信号满足180秒最小间隔

窗口约束验证（前100个保留信号）:
  ✓ 前100个保留信号都满足窗口约束

===== 信号密度 =====
保留信号时间跨度: 167,562,103秒
平均信号间隔: 762.4秒
信号密度: 4.72个/小时

计算时间: 3330.48秒

===== 高分样本展示 =====
                open_time    close  long_score  long_holding_time
0     2019-12-31 16:00:00  

In [ ]:
df_with_scores.head()

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,long_score,long_base_score,long_trade_result,long_holding_time,long_high_score_kept
0,2019-12-31 16:00:00,7195.00,7195.00,7194.91,7194.91,0.255913,2019-12-31 16:00:00.999,1841.277907,7,0.000000,0.000000,79.247842,79.247842,2.0,92.0,True
1,2019-12-31 16:00:01,7194.27,7195.61,7194.26,7194.28,0.449301,2019-12-31 16:00:01.999,3232.616972,6,0.245360,1765.410655,0.000000,79.255863,2.0,91.0,False
2,2019-12-31 16:00:02,7195.57,7195.57,7195.57,7195.57,0.006718,2019-12-31 16:00:02.999,48.339839,2,0.006718,48.339839,0.000000,79.263888,2.0,90.0,False
3,2019-12-31 16:00:03,7194.28,7195.57,7194.28,7194.28,0.032694,2019-12-31 16:00:03.999,235.220110,4,0.008000,57.564560,0.000000,79.271917,2.0,89.0,False
4,2019-12-31 16:00:04,7194.28,7195.56,7194.28,7194.97,0.292791,2019-12-31 16:00:04.999,2106.670158,3,0.083160,598.382770,0.000000,79.279948,2.0,88.0,False


In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from scipy import stats

# # ============================================
# # 评分时间分布分析
# # ============================================

# def prepare_time_features(df, score_col='long_score'):
#     """
#     准备时间特征列
#     """
#     df_analysis = df.copy()

#     # 确保open_time是datetime类型
#     if df_analysis['open_time'].dtype != 'datetime64[ns]':
#         df_analysis['open_time'] = pd.to_datetime(df_analysis['open_time'])

#     # 提取时间特征
#     df_analysis['year'] = df_analysis['open_time'].dt.year
#     df_analysis['month'] = df_analysis['open_time'].dt.month
#     df_analysis['day'] = df_analysis['open_time'].dt.day
#     df_analysis['hour'] = df_analysis['open_time'].dt.hour
#     df_analysis['weekday'] = df_analysis['open_time'].dt.dayofweek  # 0=Monday, 6=Sunday
#     df_analysis['date'] = df_analysis['open_time'].dt.date

#     # 创建时间段分类（4小时一段）
#     df_analysis['time_period'] = pd.cut(df_analysis['hour'],
#                                         bins=[0, 4, 8, 12, 16, 20, 24],
#                                         labels=['0-4h', '4-8h', '8-12h', '12-16h', '16-20h', '20-24h'],
#                                         include_lowest=True)

#     # 创建分数区间（10分一档） - 修复版本
#     # 先创建一个包含所有可能类别的分箱
#     bins = [-0.1, 0.1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
#     labels = ['0(SL)', '1-10', '10-20', '20-30', '30-40', '40-50',
#               '50-60', '60-70', '70-80', '80-90', '90-100']

#     df_analysis['score_bin'] = pd.cut(df_analysis[score_col],
#                                       bins=bins,
#                                       labels=labels,
#                                       include_lowest=True)

#     return df_analysis


# def analyze_time_distribution(df_analysis, score_col='long_score'):
#     """
#     分析时间分布的均匀性
#     """
#     print("="*80)
#     print("Time Distribution Analysis")
#     print("="*80)

#     # 1. 年度分布
#     print("\n1. Yearly Distribution:")
#     print("-" * 40)
#     yearly_dist = df_analysis.groupby(['year', 'score_bin']).size().unstack(fill_value=0)
#     print(yearly_dist)

#     # 计算年度分布的变异系数
#     for score_range in yearly_dist.columns:
#         cv = yearly_dist[score_range].std() / yearly_dist[score_range].mean() if yearly_dist[score_range].mean() > 0 else 0
#         print(f"  {score_range}: CV={cv:.2f} {'(Non-uniform)' if cv > 0.5 else '(Relatively uniform)'}")

#     # 2. 月度分布
#     print("\n2. Monthly Distribution (All Years Combined):")
#     print("-" * 40)
#     monthly_dist = df_analysis.groupby(['month', 'score_bin']).size().unstack(fill_value=0)

#     # 只显示关键分数段
#     key_bins = ['0(SL)', '60-70', '70-80', '80-90', '90-100']
#     available_bins = [b for b in key_bins if b in monthly_dist.columns]
#     if available_bins:
#         print(monthly_dist[available_bins])

#         # 卡方检验 - 检验月度分布是否均匀
#         for score_range in available_bins:
#             chi2, p_value = stats.chisquare(monthly_dist[score_range])
#             print(f"  {score_range}: p-value={p_value:.4f} {'(Non-uniform)' if p_value < 0.05 else '(Uniform)'}")

#     # 3. 星期分布
#     print("\n3. Weekday Distribution:")
#     print("-" * 40)
#     weekday_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
#     weekday_dist = df_analysis.groupby(['weekday', 'score_bin']).size().unstack(fill_value=0)
#     weekday_dist.index = weekday_names

#     if available_bins:
#         print(weekday_dist[available_bins])

#     # 4. 小时分布（时段）
#     print("\n4. Time Period Distribution (24 hours):")
#     print("-" * 40)
#     hourly_dist = df_analysis.groupby(['time_period', 'score_bin']).size().unstack(fill_value=0)

#     if available_bins:
#         print(hourly_dist[available_bins])

#         # 计算时段分布的标准差
#         for score_range in available_bins:
#             std_dev = hourly_dist[score_range].std()
#             mean_val = hourly_dist[score_range].mean()
#             cv = std_dev / mean_val if mean_val > 0 else 0
#             print(f"  {score_range}: Mean={mean_val:.0f}, StdDev={std_dev:.0f}, CV={cv:.2f}")

#     # 5. 高分时段识别
#     print("\n5. High Score Concentration (>=90):")
#     print("-" * 40)
#     high_scores = df_analysis[df_analysis[score_col] >= 90]
#     if len(high_scores) > 0:
#         # 按小时统计
#         hourly_high = high_scores.groupby('hour').size().sort_values(ascending=False).head(5)
#         print("Top 5 Hours:")
#         for hour, count in hourly_high.items():
#             print(f"  {hour:02d}:00 - {count:,} high scores")

#         # 按日期统计
#         daily_high = high_scores.groupby('date').size().sort_values(ascending=False).head(5)
#         print("\nTop 5 Dates:")
#         for date, count in daily_high.items():
#             print(f"  {date} - {count:,} high scores")

#     return df_analysis


# def plot_time_distribution(df_analysis, score_col='long_score', save_plots=False):
#     """
#     可视化时间分布
#     """
#     fig, axes = plt.subplots(2, 2, figsize=(15, 10))

#     # 1. 月度热力图
#     monthly_pivot = df_analysis.pivot_table(
#         values=score_col,
#         index='month',
#         columns='hour',
#         aggfunc='mean'
#     )
#     sns.heatmap(monthly_pivot, ax=axes[0, 0], cmap='YlOrRd', cbar_kws={'label': 'Average Score'})
#     axes[0, 0].set_title('Monthly-Hourly Average Score Heatmap')
#     axes[0, 0].set_xlabel('Hour')
#     axes[0, 0].set_ylabel('Month')

#     # 2. 高分(>=70)占比的时间分布
#     df_analysis['is_high_score'] = (df_analysis[score_col] >= 70).astype(int)
#     hourly_ratio = df_analysis.groupby('hour')['is_high_score'].mean() * 100
#     axes[0, 1].bar(hourly_ratio.index, hourly_ratio.values, color='steelblue')
#     axes[0, 1].set_title('High Score (>=70) Ratio by Hour')
#     axes[0, 1].set_xlabel('Hour')
#     axes[0, 1].set_ylabel('High Score Ratio (%)')
#     axes[0, 1].grid(axis='y', alpha=0.3)

#     # 3. 止损率的时间分布
#     df_analysis['is_stop_loss'] = (df_analysis[score_col] == 0).astype(int)
#     daily_sl_ratio = df_analysis.groupby('weekday')['is_stop_loss'].mean() * 100
#     weekday_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
#     axes[1, 0].bar(range(7), daily_sl_ratio.values, color='coral')
#     axes[1, 0].set_xticks(range(7))
#     axes[1, 0].set_xticklabels(weekday_names)
#     axes[1, 0].set_title('Stop Loss Rate by Weekday')
#     axes[1, 0].set_ylabel('Stop Loss Rate (%)')
#     axes[1, 0].grid(axis='y', alpha=0.3)

#     # 4. 分数分布箱线图（按时段）
#     df_analysis.boxplot(column=score_col, by='time_period', ax=axes[1, 1])
#     axes[1, 1].set_title('Score Distribution by Time Period')
#     axes[1, 1].set_xlabel('Time Period')
#     axes[1, 1].set_ylabel('Score')
#     plt.sca(axes[1, 1])
#     plt.xticks(rotation=45)

#     plt.tight_layout()

#     if save_plots:
#         plt.savefig('/kaggle/working/time_distribution_analysis.png', dpi=100, bbox_inches='tight')

#     plt.show()


# def calculate_distribution_metrics(df_analysis, score_col='long_score'):
#     """
#     计算分布均匀性指标
#     """
#     print("\n" + "="*80)
#     print("Distribution Uniformity Metrics")
#     print("="*80)

#     results = {}

#     # 1. 基尼系数 - 衡量分布不均匀程度
#     def gini_coefficient(x):
#         """计算基尼系数"""
#         sorted_x = np.sort(x)
#         n = len(x)
#         cumsum = np.cumsum(sorted_x)
#         return (2 * np.sum((np.arange(1, n+1)) * sorted_x)) / (n * np.sum(sorted_x)) - (n + 1) / n

#     # 按小时计算各分数段的分布
#     print("\n1. Gini Coefficient (0=perfectly uniform, 1=perfectly concentrated):")
#     score_bins = ['0(SL)', '60-70', '70-80', '80-90', '90-100']

#     for score_bin in score_bins:
#         if score_bin in df_analysis['score_bin'].unique():
#             hourly_counts = df_analysis[df_analysis['score_bin'] == score_bin].groupby('hour').size()
#             # 确保所有24小时都有值
#             hourly_counts = hourly_counts.reindex(range(24), fill_value=0)
#             gini = gini_coefficient(hourly_counts.values)
#             results[f'gini_{score_bin}'] = gini
#             print(f"  {score_bin}: {gini:.3f}")

#     # 2. 熵 - 衡量分布的混乱程度
#     print("\n2. Normalized Entropy (0=concentrated, 1=uniform):")
#     for score_bin in score_bins:
#         if score_bin in df_analysis['score_bin'].unique():
#             hourly_counts = df_analysis[df_analysis['score_bin'] == score_bin].groupby('hour').size()
#             hourly_counts = hourly_counts.reindex(range(24), fill_value=0)
#             # 计算概率分布
#             probs = hourly_counts / hourly_counts.sum()
#             probs = probs[probs > 0]  # 去除0概率
#             # 计算熵
#             entropy = -np.sum(probs * np.log2(probs))
#             # 归一化（除以最大可能熵）
#             max_entropy = np.log2(24)
#             normalized_entropy = entropy / max_entropy if max_entropy > 0 else 0
#             results[f'entropy_{score_bin}'] = normalized_entropy
#             print(f"  {score_bin}: {normalized_entropy:.3f}")

#     # 3. 峰度和偏度
#     print("\n3. Distribution Shape Metrics:")
#     hourly_avg_scores = df_analysis.groupby('hour')[score_col].mean()
#     skewness = stats.skew(hourly_avg_scores)
#     kurtosis = stats.kurtosis(hourly_avg_scores)
#     print(f"  Skewness: {skewness:.3f} {'(Right-skewed)' if skewness > 0 else '(Left-skewed)'}")
#     print(f"  Kurtosis: {kurtosis:.3f} {'(Peaked)' if kurtosis > 0 else '(Flat)'}")

#     return results


# # ============================================
# # 使用示例
# # ============================================

# # 假设你已经有了带评分的数据 df_with_details
# print("Starting time distribution analysis...")

# # 1. 准备数据
# df_time_analysis = prepare_time_features(df_with_details, score_col='long_score')

# # 2. 执行分布分析
# df_time_analysis = analyze_time_distribution(df_time_analysis, score_col='long_score')

# # 3. 计算均匀性指标
# metrics = calculate_distribution_metrics(df_time_analysis, score_col='long_score')

# # 4. 可视化（可选）
# plot_time_distribution(df_time_analysis, score_col='long_score', save_plots=False)

# # 5. 快速汇总
# print("\n" + "="*80)
# print("Quick Summary")
# print("="*80)

# # 找出分布最不均匀的时间维度
# print("\nMost non-uniform dimension:")
# yearly_cv = df_time_analysis.groupby(['year', 'score_bin']).size().unstack(fill_value=0).std(axis=0) / \
#            df_time_analysis.groupby(['year', 'score_bin']).size().unstack(fill_value=0).mean(axis=0)

# print(f"Score range with highest yearly variation: {yearly_cv.idxmax()} (CV={yearly_cv.max():.2f})")

# # 高分最集中的时间
# high_score_hours = df_time_analysis[df_time_analysis['long_score'] >= 70].groupby('hour').size()
# print(f"\nHour with most high scores (>=70): {high_score_hours.idxmax():02d}:00 ({high_score_hours.max():,})")
# print(f"Hour with least high scores (>=70): {high_score_hours.idxmin():02d}:00 ({high_score_hours.min():,})")

# # 提供改进建议
# print("\n" + "="*80)
# print("Recommendations")
# print("="*80)

# if metrics.get('gini_90-100', 0) > 0.3:
#     print("Warning: 90-100 score time distribution is non-uniform (Gini>0.3), consider:")
#     print("  - Check if certain periods have market characteristics causing high score concentration")
#     print("  - Consider adjusting scoring parameters by time period")

# if yearly_cv.max() > 1.0:
#     print(f"Warning: {yearly_cv.idxmax()} score range has extreme yearly variation (CV>1.0), consider:")
#     print("  - Check for market anomalies in specific years")
#     print("  - Consider training separate models by year")